In [13]:
from omegaconf import OmegaConf
import os
import polars as pl
import pandas as pd
from omegaconf import OmegaConf


In [15]:
def load_data_paths(path_type):
    """Load dataset paths from config.yaml based on the given path type.

    Parameters
    ----------
    path_type : str
        Either 'in_paths' for input files or 'out_paths' for output files.

    Returns
    -------
    dict
        a dictionary where each key is dataset name, each value is its full file path.
    Raises
    ------
    ValueError
        If 'path_type' is not found in the config file.
    """
    current_path=os.getcwd()
    dir=os.path.abspath(os.path.join(current_path, os.pardir))
    config_path = os.path.join(dir, "config.yaml")
    config = OmegaConf.load(config_path)
    if path_type not in config:
        raise ValueError(
            f"Invalid path_type '{path_type}'. Choose 'in_paths' or 'out_paths'."
        )

    paths_dict = config[path_type]
    data_paths_dict = {
        key: os.path.join(dir, value).replace("\\", "/")
        for key, value in paths_dict.items()
    }
    return data_paths_dict


In [18]:
load_data_paths("out_paths")

{'dos_df': 'c:/Users/Naz/Documents/GitHub/Newky/IDS/Intrusion-Detection-Systems-using-ML/output/dos_df.csv',
 'fuzzy_df': 'c:/Users/Naz/Documents/GitHub/Newky/IDS/Intrusion-Detection-Systems-using-ML/output/fuzzy_df.csv',
 'attack_free_df': 'c:/Users/Naz/Documents/GitHub/Newky/IDS/Intrusion-Detection-Systems-using-ML/output/attack_free_df.csv'}